In [44]:
import matplotlib.pyplot as plt
import pandas as pd

In [45]:
%matplotlib inline
a=[100000000,200000000,30000000000]
b=[1,2,3]
df=pd.DataFrame([a,b])
df.columns=['c','a','o']
df.index=['j','i']

In [46]:
df

,c,a,o
j,100000000,200000000,30000000000
i,1,2,3
